# **Animals in Shelter Web Scraper**



To scrape the data given the website i've choosen, the process will have 5 main sections:

1. Preliminar evaluation
2. Reach to the website 
3. Get url from each pet's site
4. Scrape data from each pet's site
5. Gather all data on csv

In [ ]:
#Installation of required packages and libraries for the project

!pip install selenium
!pip install python-whois
!pip install tqdm
!pip install requests-html
import builtwith
import whois
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
from time import sleep
import requests
import random
import time
import pandas as pd

### **Preliminar evaluation**

Previous evaluation, we will inspect Robots.txt, the technology used to build the website and find out the website owner, i'm using 'benestaranimal site' from 'Ajuntament de Barcelona', this site is both in spanish and catalan this is defined by the "ca" or "es" after the "/"at the end of the site. To evaluate Robots.txt i will have to use the roo url  meaning i have to reach https://ajuntament.barcelona.cat/benestaranimal/robots.txt

In [ ]:
'''
print ("technology used to develope the website i'm gonna use")
print(builtwith.builtwith('https://ajuntament.barcelona.cat/benestaranimal/ca'))
print ("owner of the website")
print(whois.whois('https://ajuntament.barcelona.cat/benestaranimal/ca'))
print('robots.txt detail')
print(requests.get('https://ajuntament.barcelona.cat/benestaranimal/robots.txt', data=None).text)
'''

### **Reach to website**

Will have as a result that we can open the website with webdriver in selenium to navigate and get to the specific page where all data from the different animals is presented.

In [ ]:

url="https://ajuntament.barcelona.cat/benestaranimal/ca"

driver=webdriver.Chrome()

driver_setting=webdriver.ChromeOptions()
driver_setting.add_argument("user-agent=Study_purpose_crawler")

driver.get(url)

#i'm putting some delay between clicking buttons just to make sure the page is already loaded.

time.sleep(3)

driver.find_element_by_xpath('//*[@id="main-menu-ul"]/li[4]/span').click()

time.sleep(2)

driver.find_element_by_xpath('//*[@id="main-menu-ul"]/li[4]/div/ul/li[2]/ul/li[2]/ul/li[3]/a').click()


### **Get url from each pet's site**


Get all the links from each pet's site on the website since the data that we want to scrape is not on the main search page.

In [ ]:

pets_links=[]
condition=True
while condition:
    pets= driver.find_elements_by_css_selector('.views-row')
    time.sleep(10)
    for pet  in pets:
        pets1=pet.find_element_by_class_name('field-content')
        pet2=pets1.find_element_by_tag_name('a')
        pets_links.append(pet2.get_property('href'))
    try:
                
        driver.find_element_by_xpath("//a[contains(text(),'›')]").click()
        time.sleep(10)
    except:
        condition=False

# check with len function if we have got more links.

len(pets_links)    

### **Scrape data for each pet**


Get the data from each pet on the shelter

In [ ]:

alldata=[]

for pet_link in tqdm(pets_links):
    driver.get(pet_link)
    time.sleep(random.uniform(4.0,5.0))
    
    name= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[1]/h2').text
    animal_type= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[1]').text
    race= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[2]').text
    cross_race=driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()-4]').text
    sex= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()-3]').text
    age= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()-2]').text
    weight= driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()-1]').text
    id_animal=driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[2]/div[2]/div[last()]').text
    link_picture=driver.find_element_by_xpath('//*[contains(@id,"node-")]/div[2]/div/div[1]/div/div[2]/div/img').get_attribute('src')
    data= {'name':name,'Type':animal_type,'race':race,'cross_race':cross_race,'sex':sex,'age':age,'weight':weight,'id':id_animal,'link_picture':link_picture,'link_pet_site':pet_link}
    alldata.append(data)
    
driver.quit()

### **Gather all data on csv**


Gather all the data on a dataframe and convert to csv, and print result to check if the result is as expected.

In [ ]:

df=pd.DataFrame(alldata)

#Save as a csv:
df.to_csv (r'C:\Users\mila_\Documents\Master ciencia de dades\Tipología y ciclo de vida de los datos\PRAC 1\pets_in_shelter.csv', index = False, header=True)
print(df)